In [16]:
from igraph import Graph, summary
from functools import partial
from multiprocessing.pool import Pool

In [17]:
def clique_percolation_method(graph, k = 3, workers = 1, attribute = None):
    """
    graph: igraph object
    k: size of the clique
    workers: number of threads
    """
    communities  = list()
    
    cliques = graph.cliques(min=k, max=k)
    num_cliques = len(cliques)
    set_cliques = [set(i) for i in cliques]
    indices = list(range(num_cliques))

    edge_list = list()
    minimum = k-1
    annotate = partial(partial_clique_graph, set_cliques=set_cliques, minimum=minimum, num_cliques=num_cliques)
    
    pool = Pool(workers)
    edges = pool.map(annotate, indices)
    edge_list = [j for i in edges for j in i]
    
    clique_graph = Graph(edge_list)
    clique_graph.vs["name"] = [i for i in range(0,num_cliques)]
    
    components = clique_graph.decompose()
    
    for component in components:
        members_list = [list(cliques[i["name"]]) for i in component.vs] 
        this_community = [item for sublist in members_list for item in sublist]
        communities.append(list(set(this_community)))
    
    if attribute is not None:
        communities_with_names = list()
        for community in communities:
            communities_with_names.append([graph.vs[element][attribute] for element in community])
        communities = communities_with_names
    
    lens = []
    for comm in communities:
        lens.append(len(comm))
    print(lens)
        
    return communities


In [18]:
def partial_clique_graph(i, set_cliques, minimum, num_cliques):
    edge_list = list()
    this_set = set_cliques[i]
    for j in range(i+1, num_cliques):
        if len(this_set.intersection(set_cliques[j])) == minimum:
            edge_list.append((i,j))
    return edge_list    

In [ ]:
g = Graph()
vertices = set()
edges = []

with open("filename.txt", "r") as file:
    for line in file:
        vertex1, vertex2 = line.split()

        vertices.add(vertex1)
        vertices.add(vertex2)
        edges.append((vertex1, vertex2))
g.add_vertices(list(vertices))
g.add_edges(edges)
summary(g)
communities = clique_percolation_method(g,3)

In [20]:
print(len(communities))

2


In [22]:
res_dict = {}

for i, node_list in enumerate(communities):
    for node in node_list:
        res_dict[node] = i
community_list = [res_dict.get(node) for node in range(len(res_dict))]


In [23]:
for i, el in enumerate(community_list):
  if el == None:
    community_list[i] = 0

In [ ]:
lbls = []
for count, comm in enumerate(communities):
    for i in comm:
      lbls.append(g.vs[i]["name"])


In [39]:
with open('f2.txt', 'r') as f:
  file_data = f.readlines()

lbls_true = []
for line in file_data:
    lbls_true.append(line.strip().split()[1])

In [44]:
from sklearn.metrics.cluster import normalized_mutual_info_score
nmi_score_gnn = normalized_mutual_info_score(lbls_true, lbls)

In [45]:
nmi_score_gnn

0.6620536205547629

In [46]:
from sklearn.metrics import adjusted_rand_score
from sklearn.metrics import f1_score
from sklearn.metrics import jaccard_score

ari = adjusted_rand_score(lbls_true, lbls)
print(f'Adjusted Rand Index: {ari}')

f1 = f1_score(lbls_true, lbls, average='weighted')
print(f'F1 Score: {f1}')

jaccard = jaccard_score(lbls_true, lbls, average='weighted')
print(f'Jaccard Index: {jaccard}')


Adjusted Rand Index: -1.4014047367372103e-05
F1 Score: 0.0025974025974025974
Jaccard Index: 0.0013227513227513227
